## Imports

In [1]:
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
from datetime import datetime, timedelta
import glob
from tqdm.notebook import tqdm
import requests
import time

## Read-in

In [2]:
# Get today's date
today = datetime.now()

today = today - timedelta(days=1)

# Get the date one week ago
one_week_ago = today - timedelta(days=7)

# Function to format the date with the correct ordinal suffix
def format_date_with_ordinal(date):
    day = date.day
    if 4 <= day <= 20 or 24 <= day <= 30:
        suffix = "th"
    else:
        suffix = ["st", "nd", "rd"][day % 10 - 1]
    return date.strftime(f"%b. {day}{suffix}")

# Format both dates
week_ending = format_date_with_ordinal(today)
week_starting = format_date_with_ordinal(one_week_ago)

print(week_starting, week_ending)

Mar. 23rd Mar. 30th


In [3]:
filename = glob.glob('*redfin*.csv')
df_list = []
for file in filename:
    df = pd.read_csv(file)
    df_list.append(df)
    
df = pd.concat(df_list)

In [4]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [5]:
remove_string = 'In accordance with local MLS rules, some MLS listings are not included in the download'
df = df[~df['SALE TYPE'].str.contains(remove_string)]

In [6]:
df.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE'],
      dtype='object')

In [7]:
df.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
1,PAST SALE,March-24-2025,Condo/Co-op,6001 SW 70th St #305,South Miami,FL,33143.0,375000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/South-Miami/6001-SW-...,MARMLS,A11760834,N,Y,25.706558,-80.291032


In [8]:
# new_row = {'SALE TYPE':'PAST SALE','SOLD DATE':'PLACEHOLDER',
#           'PROPERTY TYPE':'Condo/Co-op',
#           'ADDRESS':'PLACEHOLDER',
#           'CITY':'PLACEHOLDER',
#           'STATE OR PROVINCE':'FL',
#           'ZIP OR POSTAL CODE':'PLACEHOLDER',
#           'PRICE':99999999999,
#           'BEDS':9999999999,
#           'BATHS':999999999,
#           'LOCATION':'PLACEHOLDER',
#           'SQUARE FEET':9999999999,
#           'LOT SIZE':None,
#           'YEAR BUILT':'PLACEHOLDER',
#           'DAYS ON MARKET':None,
#           '$/SQUARE FEET':9999999999,
#           'HOA/MONTH':999999999,
#           'STATUS':'Sold',
#           'NEXT OPEN HOUSE START TIME':'PLACEHOLDER',
#           'NEXT OPEN HOUSE END TIME':'PLACEHOLDER',
#           'URL':'Manual Entry',
#           'SOURCE':'Manual Entry',
#           'MLS#':'Manual Entry',
#           'FAVORITE':'N',
#           'INTERESTED':'Y',
#           'LATITUDE':99.999999999,
#           'LONGITUDE':99.999999999}

In [8]:
df['SOLD DATE'] = pd.to_datetime(df['SOLD DATE'])

In [9]:
df2 = df[(df['SOLD DATE'] > one_week_ago) & (df['SOLD DATE'] < today)]

In [10]:
df2['SOLD DATE'].value_counts()

SOLD DATE
2025-03-28    31
2025-03-24    29
2025-03-25    29
2025-03-27    20
2025-03-26    17
Name: count, dtype: int64

In [11]:
df2 = df2.sort_values(by='PRICE',ascending=False)

In [69]:
# df2 = df2.drop(index=64)

In [12]:
df_top_ten = df2.head(10)

In [13]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
74,PAST SALE,2025-03-24,Condo/Co-op,321 Ocean Dr #201,Miami Beach,FL,33139.0,11250000.0,5.0,5.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/321-Ocea...,MARMLS,A11695945,N,Y,25.772296,-80.131901
118,PAST SALE,2025-03-28,Condo/Co-op,17749 Collins Ave #802,Sunny Isles Beach,FL,33160.0,6100000.0,4.0,6.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11611470,N,Y,25.940065,-80.120264
126,PAST SALE,2025-03-28,Condo/Co-op,17141 Collins Ave #1501,Sunny Isles Beach,FL,33160.0,4950000.0,3.0,3.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11490517,N,Y,25.934939,-80.120918
112,PAST SALE,2025-03-24,Condo/Co-op,18975 Collins Ave #2303,Sunny Isles Beach,FL,33160.0,3800000.0,3.0,3.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,Beaches MLS,RX-11004551,N,Y,25.951740,-80.119755
52,PAST SALE,2025-03-26,Condo/Co-op,18975 Collins Ave #903,Sunny Isles Beach,FL,33160.0,3775000.0,3.0,3.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,MARMLS,A11712041,N,Y,25.951740,-80.119755
63,PAST SALE,2025-03-27,Condo/Co-op,300 S Pointe Dr #1105,Miami Beach,FL,33139.0,3600000.0,3.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/300-S-Po...,MARMLS,A11710839,N,Y,25.767706,-80.134877
122,PAST SALE,2025-03-24,Condo/Co-op,2669 S Bayshore Dr Unit 1602N,Miami,FL,33133.0,3500000.0,3.0,3.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2669-S-Bayshor...,MARMLS,A11582132,N,Y,25.730676,-80.237117
120,PAST SALE,2025-03-28,Condo/Co-op,1800 Sunset Harbour Dr Unit 1102/04,Miami Beach,FL,33139.0,2837500.0,4.0,3.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/1800-Sun...,MARMLS,A11606692,N,Y,25.794488,-80.144801
41,PAST SALE,2025-03-24,Condo/Co-op,18975 Collins Ave #604,Sunny Isles Beach,FL,33160.0,2600000.0,2.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,MARMLS,A11720744,N,Y,25.951740,-80.119755
117,PAST SALE,2025-03-25,Condo/Co-op,749 Crandon Blvd #212,Key Biscayne,FL,33149.0,2575000.0,4.0,3.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Key-Biscayne/749-Cra...,MARMLS,A11618404,N,Y,25.687221,-80.161390


In [12]:
# df_top_ten = df2.head(10)

## Collect Agent Information

In [14]:
def process_response(url):
    try:
        
        base_url = "https://redfin-com-data.p.rapidapi.com/properties/detail-by-url"

        querystring = {"url":url}

        headers = {
            "x-rapidapi-key": "00191da588msh8450293d26e3515p1bbd40jsn56510b513b61",
            "x-rapidapi-host": "redfin-com-data.p.rapidapi.com"
        }

        response = requests.get(base_url, headers=headers, params=querystring)

#         print(response.status_code)
#         print(url)
        time.sleep(0.2)

        response.raise_for_status()
        data = response.json()

        # Initialize default values for listing and buyer agents data
        list_agent_name_1 = None
        list_broker_name_1 = None
        list_agent_name_2 = None
        list_broker_name_2 = None
        
        buy_agent_name_1 = None
        buy_broker_name_1 = None
        buy_agent_name_2 = None
        buy_broker_name_2 = None
        
        # Extract listing agents data if it exists
        listing_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('listingAgents', [])
        if len(listing_agents) > 0:
            list_agent_name_1 = listing_agents[0].get('agentInfo', {}).get('agentName')
            list_broker_name_1 = listing_agents[0].get('brokerName')
        if len(listing_agents) > 1:
            list_agent_name_2 = listing_agents[1].get('agentInfo', {}).get('agentName')
            list_broker_name_2 = listing_agents[1].get('brokerName')
            
        # Extract listing agents data if it exists
        buying_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('buyingAgents', [])
        if len(buying_agents) > 0:
            buy_agent_name_1 = buying_agents[0].get('agentInfo', {}).get('agentName')
            buy_broker_name_1 = buying_agents[0].get('brokerName')
        if len(buying_agents) > 1:
            buy_agent_name_2 = buying_agents[1].get('agentInfo', {}).get('agentName')
            buy_broker_name_2 = buying_agents[1].get('brokerName')


        return {
            'url': url,
            'list_agent_name_1': list_agent_name_1,
            'list_broker_name_1': list_broker_name_1,
            'list_agent_name_2': list_agent_name_2,
            'list_broker_name_2': list_broker_name_2,
            'buy_agent_name_1': buy_agent_name_1,
            'buy_broker_name_1': buy_broker_name_1,
            'buy_agent_name_2': buy_agent_name_2,
            'buy_broker_name_2': buy_broker_name_2,
        }
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from {url}: {e}")
        return {
            'url': url,
            'list_agent_name_1': None,
            'list_broker_name_1': None,
            'list_agent_name_2': None,
            'list_broker_name_2': None,
            'buy_agent_name_1': None,
            'buy_broker_name_1': None,
            'buy_agent_name_2': None,
            'buy_broker_name_2': None,
        }

In [15]:
response_list = []

for index, row in df_top_ten.iterrows():
    response_list.append(process_response(row['URL']))

In [16]:
response_df = pd.DataFrame(response_list)

In [17]:
merged_df = pd.merge(left=df_top_ten,left_on='URL',right=response_df, right_on='url')

In [18]:
df_top_ten = merged_df

In [19]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2
0,PAST SALE,2025-03-24,Condo/Co-op,321 Ocean Dr #201,Miami Beach,FL,33139.0,11250000.0,5.0,5.5,...,-80.131901,https://www.redfin.com/FL/Miami-Beach/321-Ocea...,Michael Duchon,The Corcoran Group,Eloy Carmenate,The Corcoran Group,Darin Tansey,Douglas Elliman,None,None
1,PAST SALE,2025-03-28,Condo/Co-op,17749 Collins Ave #802,Sunny Isles Beach,FL,33160.0,6100000.0,4.0,6.5,...,-80.120264,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,Adi Amuial,Miles Goldstein Real Estate,None,None,Adriana Vargas Hernandez,Acqualina Realty,None,None
2,PAST SALE,2025-03-28,Condo/Co-op,17141 Collins Ave #1501,Sunny Isles Beach,FL,33160.0,4950000.0,3.0,3.5,...,-80.120918,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,Roberta Poli,"Related ISG Realty, LLC.",None,None,Viktoriya Donisova Ho,"Compass Florida, LLC.",None,None
3,PAST SALE,2025-03-24,Condo/Co-op,18975 Collins Ave #2303,Sunny Isles Beach,FL,33160.0,3800000.0,3.0,3.5,...,-80.119755,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,Karine Frangulyan,Signature One Luxury Estates LLC,None,None,Wendy Cohen,One Sotheby's International Realty,None,None
4,PAST SALE,2025-03-26,Condo/Co-op,18975 Collins Ave #903,Sunny Isles Beach,FL,33160.0,3775000.0,3.0,3.5,...,-80.119755,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,Roger Silva,The Florida Lounge Realty LLC,None,None,Romina Grinberg,"Compass Florida, LLC",None,None
5,PAST SALE,2025-03-27,Condo/Co-op,300 S Pointe Dr #1105,Miami Beach,FL,33139.0,3600000.0,3.0,2.5,...,-80.134877,https://www.redfin.com/FL/Miami-Beach/300-S-Po...,None,None,None,None,None,None,None,None
6,PAST SALE,2025-03-24,Condo/Co-op,2669 S Bayshore Dr Unit 1602N,Miami,FL,33133.0,3500000.0,3.0,3.5,...,-80.237117,https://www.redfin.com/FL/Miami/2669-S-Bayshor...,Ivan Madera,Fortune Christie's International Real Estate,None,None,Lindsey Lamchick,The Edit,None,None
7,PAST SALE,2025-03-28,Condo/Co-op,1800 Sunset Harbour Dr Unit 1102/04,Miami Beach,FL,33139.0,2837500.0,4.0,3.5,...,-80.144801,https://www.redfin.com/FL/Miami-Beach/1800-Sun...,Paul Sasseville,"Compass Florida, LLC.",None,None,Cory Waldman,Coldwell Banker Realty,None,None
8,PAST SALE,2025-03-24,Condo/Co-op,18975 Collins Ave #604,Sunny Isles Beach,FL,33160.0,2600000.0,2.0,2.5,...,-80.119755,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,Luciene Cofresi,Fortune International Realty,None,None,Claudio Minones,One Sotheby's International Realty,None,None
9,PAST SALE,2025-03-25,Condo/Co-op,749 Crandon Blvd #212,Key Biscayne,FL,33149.0,2575000.0,4.0,3.0,...,-80.161390,https://www.redfin.com/FL/Key-Biscayne/749-Cra...,Gloria Ibarra,Realty Progress LLC,None,None,Natalia Salgado,DLS International Realty,None,None


## Current Week's Values

In [20]:
# Number of condos sold
current_week_condo_sales = len(df2)
print(len(df2))

126


In [21]:
# Average sale price
average_sale_price = round(df2['PRICE'].mean())
print(f'${average_sale_price:,.0f}')

$843,850


In [22]:
# Average PSF
average_price_psf = round(df2['$/SQUARE FEET'].mean())
print(f'${average_price_psf:,.0f}')

$547


In [23]:
# Total sales dollar volume
current_week_condo_sales_total = df2['PRICE'].sum()
print(f'${current_week_condo_sales_total:,.0f}')

$106,325,153


In [24]:
def load_last_week_data_df(filename='last_week_data.csv'):
    try:
        return pd.read_csv(filename)
    except FileNotFoundError:
        return None
    
last_week = load_last_week_data_df()

In [25]:
print('Input county name:')
county = input()
print('Input Previous Week Condo Sales Total (number units sold):')
# previous_week_condo_sales = input()
previous_week_condo_sales = last_week['Condo Sales'].iloc[0]
print(previous_week_condo_sales)
print('-----------')

print('Input Previous Week Condo Average Sales Price:')
# previous_week_average_sales_price = input()
previous_week_average_sales_price = last_week['Average Sale Price'].iloc[0]
print(previous_week_average_sales_price)
print('-----------')

print('Input Previous Week Condo Average PSF:')
# previous_week_average_psf = input()
previous_week_average_psf = last_week['Average PSF'].iloc[0]
print(previous_week_average_psf)
print('-----------')

print('Input Previous Week Condo Sales total (ex: 198_000_000)')
# previous_week_condo_sales_total = input()
previous_week_condo_sales_total = last_week['Last Week Total Dollar Volume'].iloc[0]
print(previous_week_condo_sales_total)


## Clean Data

Input county name:
Miami-Dade
Input Previous Week Condo Sales Total (number units sold):
116
-----------
Input Previous Week Condo Average Sales Price:
682803
-----------
Input Previous Week Condo Average PSF:
536
-----------
Input Previous Week Condo Sales total (ex: 198_000_000)
79205150.0


In [26]:
last_week

,Condo Sales,Average Sale Price,Average PSF,Last Week Total Dollar Volume
0,116,682803,536,79205150.0


In [27]:
last_week['Last Week Total Dollar Volume'].iloc[0]

79205150.0

In [28]:
def save_current_week_data_df(df, filename='last_week_data.csv'):
    df.to_csv(filename, index=False)

# Create a dictionary of your current week data.
current_week_data = {
    "Condo Sales": current_week_condo_sales,
    "Average Sale Price": average_sale_price,
    "Average PSF": average_price_psf,
    "Last Week Total Dollar Volume": current_week_condo_sales_total
}

# Convert the dictionary into a DataFrame.
current_week_summary_df = pd.DataFrame([current_week_data])

# Save the DataFrame to a CSV file.
save_current_week_data_df(current_week_summary_df)

# Next week, load last week's data automatically.
last_week_df = load_last_week_data_df()
if last_week_df is not None:
    print("Last week's data:")
    print(last_week_df)
else:
    print("No previous data found.")

Last week's data:
   Condo Sales  Average Sale Price  Average PSF  Last Week Total Dollar Volume
0          126              843850          547                    106325153.0


## Previous Week's Values

### The below 2 cells are only used if we need to back fill from an earlier week than just last week.

For example, if we start in a new locale and comparison information can't get pulled from an existing story

In [22]:
# # Define the date range
# start_date = '2024-12-23'
# end_date = '2024-12-29'

# # Filter the DataFrame between two dates (inclusive)
# old_week = df[(df['SOLD DATE'] >= start_date) & (df['SOLD DATE'] <= end_date)]

In [21]:
# # comparison week total sales
# print(f'Comparison week filtered sales: {(len(old_week))}')
# print('-----------')
# print(f'Comparison week average sales price: ${old_week["PRICE"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week average psf: ${old_week["$/SQUARE FEET"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week sales total: ${old_week["PRICE"].sum():,.02f}')
# print('-----------')

In [36]:
# df_top_ten

## Correction section

In [10]:
# df.at[index#,'col_name']

## Format Data

In [29]:
### Insert NaNs if needed ###
df_top_ten = df_top_ten.replace('N/A', np.nan)

## Color-code top sale

In [30]:
### Insert RANK values ###
df_top_ten['RANK'] = range(1, len(df_top_ten) + 1)
# use numpy to assign values to the 'COLOR' column
df_top_ten['COLOR'] = np.where(df_top_ten['RANK'] <= 1, 'orange', 'blue')

## HTML Popup Formatter

In [31]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR'],
      dtype='object')

In [32]:
pd.set_option('display.max_columns',None)

In [33]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-03-24,Condo/Co-op,321 Ocean Dr #201,Miami Beach,FL,33139.0,11250000.0,5.0,5.5,321 OCEAN CONDO,4400.0,NaN,2015.0,NaN,2557.0,28702.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/321-Ocea...,MARMLS,A11695945,N,Y,25.772296,-80.131901,https://www.redfin.com/FL/Miami-Beach/321-Ocea...,Michael Duchon,The Corcoran Group,Eloy Carmenate,The Corcoran Group,Darin Tansey,Douglas Elliman,None,None,1,orange


In [34]:
def popup_html(row):
    address = row['ADDRESS']  
    price = row['PRICE']
    psf = row['$/SQUARE FEET']
    sf = row['SQUARE FEET']
    year_built = row['YEAR BUILT']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(address) + '''<br>
    <strong>Sale Price: </strong>${}'''.format(f'{price:,}') + '''<br>
    <strong>Price sq ft: </strong>${}'''.format(f'{psf:,}') + '''<br>
    <strong>Square Footage: </strong>{}'''.format(f'{sf:,}') + '''<br>
    <strong>Year Built: </strong>{}'''.format(f'{str(year_built)}') + '''<br>
    </html>
    '''
    return html

In [35]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-03-24,Condo/Co-op,321 Ocean Dr #201,Miami Beach,FL,33139.0,11250000.0,5.0,5.5,321 OCEAN CONDO,4400.0,NaN,2015.0,NaN,2557.0,28702.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/321-Ocea...,MARMLS,A11695945,N,Y,25.772296,-80.131901,https://www.redfin.com/FL/Miami-Beach/321-Ocea...,Michael Duchon,The Corcoran Group,Eloy Carmenate,The Corcoran Group,Darin Tansey,Douglas Elliman,None,None,1,orange
1,PAST SALE,2025-03-28,Condo/Co-op,17749 Collins Ave #802,Sunny Isles Beach,FL,33160.0,6100000.0,4.0,6.5,17749 COLLINS AVENUE COND,4609.0,NaN,2015.0,NaN,1323.0,10288.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11611470,N,Y,25.940065,-80.120264,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,Adi Amuial,Miles Goldstein Real Estate,None,None,Adriana Vargas Hernandez,Acqualina Realty,None,None,2,blue
2,PAST SALE,2025-03-28,Condo/Co-op,17141 Collins Ave #1501,Sunny Isles Beach,FL,33160.0,4950000.0,3.0,3.5,MUSE CONDO,3195.0,NaN,2018.0,NaN,1549.0,9176.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11490517,N,Y,25.934939,-80.120918,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,Roberta Poli,"Related ISG Realty, LLC.",None,None,Viktoriya Donisova Ho,"Compass Florida, LLC.",None,None,3,blue
3,PAST SALE,2025-03-24,Condo/Co-op,18975 Collins Ave #2303,Sunny Isles Beach,FL,33160.0,3800000.0,3.0,3.5,18975 Collins Condo,2271.0,NaN,2019.0,NaN,1673.0,3734.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,Beaches MLS,RX-11004551,N,Y,25.951740,-80.119755,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,Karine Frangulyan,Signature One Luxury Estates LLC,None,None,Wendy Cohen,One Sotheby's International Realty,None,None,4,blue
4,PAST SALE,2025-03-26,Condo/Co-op,18975 Collins Ave #903,Sunny Isles Beach,FL,33160.0,3775000.0,3.0,3.5,18975 COLLINS CONDO,2271.0,NaN,2019.0,NaN,1662.0,8904.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,MARMLS,A11712041,N,Y,25.951740,-80.119755,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,Roger Silva,The Florida Lounge Realty LLC,None,None,Romina Grinberg,"Compass Florida, LLC",None,None,5,blue
5,PAST SALE,2025-03-27,Condo/Co-op,300 S Pointe Dr #1105,Miami Beach,FL,33139.0,3600000.0,3.0,2.5,PORTOFINO TOWER CONDO,2340.0,NaN,1996.0,NaN,1538.0,4016.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/300-S-Po...,MARMLS,A11710839,N,Y,25.767706,-80.134877,https://www.redfin.com/FL/Miami-Beach/300-S-Po...,None,None,None,None,None,None,None,None,6,blue
6,PAST SALE,2025-03-24,Condo/Co-op,2669 S Bayshore Dr Unit 1602N,Miami,FL,33133.0,3500000.0,3.0,3.5,THE GROVE AT GRAND BAY,1829.0,NaN,2016.0,NaN,1914.0,3083.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2669-S-Bayshor...,MARMLS,A11582132,N,Y,25.730676,-80.237117,https://www.redfin.com/FL/Miami/2669-S-Bayshor...,Ivan Madera,Fortune Christie's International Real Estate,None,None,Lindsey Lamchick,The Edit,None,None,7,blue
7,PAST SALE,2025-03-28,Condo/Co-op,1800 Sunset Harbour Dr Unit 1102/04,Miami Beach,FL,33139.0,2837500.0,4.0,3.5,SUNSET HARBOUR SOUTH COND,2246.0,NaN,1998.0,NaN,1263.0,3317.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/1800-Sun...,MARMLS,A11606692,N,Y,25.794488,-80.144801,https://www.redfin.com/FL/Miami-Beach/1800-Sun...,Paul Sasseville,"Compass Florida, LLC.",None,None,Cory Waldman,Coldwell Banker Realty,None,None,8,blue
8,PAST SALE,2025-03-24,Condo/Co-op,18975 Collins Ave #604,Sunny Isles Beach,FL,33160.0,2600000.0,2.0,2.5,18975 COLLINS CONDO,1521.0,NaN,2019.0,NaN,1709.0,2983.0,Sold,NaN,NaN,https://www.r

In [36]:
from folium.plugins import MarkerCluster

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent {county} County Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format(f'{week_starting} — {week_ending}')


### Create map container ###
m = folium.Map(location=df[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=9.5,tiles=None)

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top Sale')

for index, row in df_top_ten.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))
            
# Display map
m

In [37]:
m.save('index.html')

## Data snagger

In [38]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [39]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-03-24,Condo/Co-op,321 Ocean Dr #201,Miami Beach,FL,33139.0,11250000.0,5.0,5.5,321 OCEAN CONDO,4400.0,NaN,2015.0,NaN,2557.0,28702.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/321-Ocea...,MARMLS,A11695945,N,Y,25.772296,-80.131901,https://www.redfin.com/FL/Miami-Beach/321-Ocea...,Michael Duchon,The Corcoran Group,Eloy Carmenate,The Corcoran Group,Darin Tansey,Douglas Elliman,None,None,1,orange


In [122]:
# # Highest and lowest sale price
# highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
# lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

# print(f"Highest Sale Price: {df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_sale_price_idx]['DAYS ON MARKET']}")
# print(f"Lowest Sale Price: {df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_sale_price_idx]['DAYS ON MARKET']}")

# # Highest and lowest psf
# highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
# lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

# print(f"Highest PSF: {df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_psf_idx]['DAYS ON MARKET']}")
# print(f"Lowest PSF: {df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_psf_idx]['DAYS ON MARKET']}")

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [40]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/MiamiDade_County_condo_sales_week_ending_03312024


# CREATE TEMPLATE 

In [41]:
muni_set = set(df_top_ten['CITY'])

In [42]:
df_top_ten['FULL_NAME'] = df_top_ten['LOCATION'] + ' at ' + df_top_ten['ADDRESS'] + ' in ' + df_top_ten['CITY']

In [43]:
df_top_ten.reset_index(inplace=True,drop=True)

In [44]:
top_sale = df_top_ten.at[0,'FULL_NAME']
second_top_sale = df_top_ten.at[1,'FULL_NAME']

In [45]:
top_sale

'321 OCEAN CONDO at 321 Ocean Dr #201 in Miami Beach'

In [46]:
### Average Sales Price ###
if float(average_sale_price) > float(previous_week_average_sales_price):
    change = 'higher'
    difference = float(average_sale_price) - float(previous_week_average_sales_price)
else:
    change = 'lower'
    difference = float(previous_week_average_sales_price) - float(average_sale_price)
    
### Total condo dollar volume ###
if float(current_week_condo_sales_total) > float(previous_week_condo_sales_total):
    seo_head = 'Rises'
elif float(previous_week_condo_sales_total) < float(current_week_condo_sales_total):
    seo_head = 'Falls'
else:
    seo_head = 'Unchanged'
    
### Average PSF ###    
if float(average_price_psf) > float(previous_week_average_psf):
    psf_rf = 'rose'
elif float(average_price_psf) < float(previous_week_average_psf):
    psf_rf ='fell'
else:
    psf_rf = 'equaled'

In [47]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [48]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-03-24,Condo/Co-op,321 Ocean Dr #201,Miami Beach,FL,33139.0,11250000.0,5.0,5.5,321 OCEAN CONDO,4400.0,NaN,2015.0,NaN,2557.0,28702.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/321-Ocea...,MARMLS,A11695945,N,Y,25.772296,-80.131901,https://www.redfin.com/FL/Miami-Beach/321-Ocea...,Michael Duchon,The Corcoran Group,Eloy Carmenate,The Corcoran Group,Darin Tansey,Douglas Elliman,None,None,1,orange,321 OCEAN CONDO at 321 Ocean Dr #201 in Miami ...
1,PAST SALE,2025-03-28,Condo/Co-op,17749 Collins Ave #802,Sunny Isles Beach,FL,33160.0,6100000.0,4.0,6.5,17749 COLLINS AVENUE COND,4609.0,NaN,2015.0,NaN,1323.0,10288.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11611470,N,Y,25.940065,-80.120264,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,Adi Amuial,Miles Goldstein Real Estate,None,None,Adriana Vargas Hernandez,Acqualina Realty,None,None,2,blue,17749 COLLINS AVENUE COND at 17749 Collins Ave...
2,PAST SALE,2025-03-28,Condo/Co-op,17141 Collins Ave #1501,Sunny Isles Beach,FL,33160.0,4950000.0,3.0,3.5,MUSE CONDO,3195.0,NaN,2018.0,NaN,1549.0,9176.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11490517,N,Y,25.934939,-80.120918,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,Roberta Poli,"Related ISG Realty, LLC.",None,None,Viktoriya Donisova Ho,"Compass Florida, LLC.",None,None,3,blue,MUSE CONDO at 17141 Collins Ave #1501 in Sunny...
3,PAST SALE,2025-03-24,Condo/Co-op,18975 Collins Ave #2303,Sunny Isles Beach,FL,33160.0,3800000.0,3.0,3.5,18975 Collins Condo,2271.0,NaN,2019.0,NaN,1673.0,3734.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,Beaches MLS,RX-11004551,N,Y,25.951740,-80.119755,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,Karine Frangulyan,Signature One Luxury Estates LLC,None,None,Wendy Cohen,One Sotheby's International Realty,None,None,4,blue,18975 Collins Condo at 18975 Collins Ave #2303...
4,PAST SALE,2025-03-26,Condo/Co-op,18975 Collins Ave #903,Sunny Isles Beach,FL,33160.0,3775000.0,3.0,3.5,18975 COLLINS CONDO,2271.0,NaN,2019.0,NaN,1662.0,8904.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,MARMLS,A11712041,N,Y,25.951740,-80.119755,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,Roger Silva,The Florida Lounge Realty LLC,None,None,Romina Grinberg,"Compass Florida, LLC",None,None,5,blue,18975 COLLINS CONDO at 18975 Collins Ave #903 ...
5,PAST SALE,2025-03-27,Condo/Co-op,300 S Pointe Dr #1105,Miami Beach,FL,33139.0,3600000.0,3.0,2.5,PORTOFINO TOWER CONDO,2340.0,NaN,1996.0,NaN,1538.0,4016.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/300-S-Po...,MARMLS,A11710839,N,Y,25.767706,-80.134877,https://www.redfin.com/FL/Miami-Beach/300-S-Po...,None,None,None,None,None,None,None,None,6,blue,PORTOFINO TOWER CONDO at 300 S Pointe Dr #1105...
6,PAST SALE,2025-03-24,Condo/Co-op,2669 S Bayshore Dr Unit 1602N,Miami,FL,33133.0,3500000.0,3.0,3.5,THE GROVE AT GRAND BAY,1829.0,NaN,2016.0,NaN,1914.0,3083.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2669-S-Bayshor...,MARMLS,A11582132,N,Y,25.730676,-80.237117,https://www.redfin.com/FL/Miami/2669-S-Bayshor...,Ivan Madera,Fortune Christie's International Real Estate,None,None,Lindsey Lamchick,The Edit,None,None,7,blue,THE GROVE AT GRAND BAY at 2669 S Bayshore Dr U...
7,PAST SALE,2025-03-28,Condo/Co-op,1800 Sunset Harbour Dr Unit 1102/04,Miami Beach,FL,33139.0,2837500.0,4.0,3.5,SUNSET HARBOUR SOUTH COND,2246.0,NaN,1998.0,NaN,1263.0,3317.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/1800-Sun...,MARMLS,A11606692,N,Y,2

In [49]:
story_string = f'''
\033[1mHED:\033[0m {df_top_ten.loc[df_top_ten['PRICE'].idxmax()]['LOCATION']} closing tops {county} weekly condo sales 
\033[1mDEK:\033[0m Top 10 sales ranged from ${df_top_ten.at[9,'PRICE']:,.0f} to ${df_top_ten.at[0,'PRICE']:,.0f}
\033[1mFEATURED HED:\033[0m
\033[1mSEO HED:\033[0m {county} County Weekly Condo Report 
\033[1mSEO DESCRIPTION:\033[0m {county} County’s condo sales XXXXXXX
\033[1mAUTHOR:\033[0m Kate Hinsche
\033[1mRESEARCH:\033[0m Adam Farence
\033[1mSocial:\033[0m #{county} condo sales XXXXX
\033[1mART:\033[0m

Image of the top sale

*Please provide credits for any images that you share
\033[1mSTORY TYPE:\033[0m Report
\033[1mSECTOR\033[0m (formerly CATEGORY): Residential Real Estate
\033[1mTAGS:\033[0m condo sales, {county}, weekly condo sales, {', '.join(str(x) for x in muni_set)}, condos, condo market

\033[1mNeighborhood:\033[0m 
\033[1mProperty:\033[0m
\033[1mProperty Type:\033[0m
\033[1mCompanies:\033[0m 
\033[1mPeople:\033[0m
\033[1mIssues:\033[0m
\033[1mRegion:\033[0m
\033[1m[]Show in Yahoo Feed\033[0m

{county} County’s NEWS PEG HERE.

Brokers closed {int(current_week_condo_sales):,.0f} condo sales totaling ${float(current_week_condo_sales_total):,.0f} million from {week_starting} to {week_ending}. The previous week, brokers closed {previous_week_condo_sales} condo sales totaling ${int(previous_week_condo_sales_total):,.0f}.

Last week’s units sold for an average of ${float(average_sale_price):,.0f}, {change} than the ${float(previous_week_average_sales_price):,.0f} average price from the previous week. The average price per square foot {psf_rf} to ${average_price_psf} from ${previous_week_average_psf}, according to data from Redfin.

For the top 10 sales, prices ranged from ${df_top_ten.at[9,'PRICE']:,} million to ${df_top_ten.at[0,'PRICE']:,} million. 

{top_sale} took the top spot with a ${df_top_ten.at[0,'PRICE']:,} million closing. The sale closed at ${df_top_ten.at[0,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[0,'list_agent_name_1']} with {df_top_ten.at[0,'list_broker_name_1']}, and {df_top_ten.at[0,'list_agent_name_2']} with {df_top_ten.at[0,'list_broker_name_2']} had the listing.
{df_top_ten.at[0,'buy_agent_name_1']} with {df_top_ten.at[0,'buy_broker_name_1']}, and {df_top_ten.at[0,'buy_agent_name_2']} with {df_top_ten.at[0,'buy_broker_name_2']} represented the buyer.

{second_top_sale}, closed for the second highest amount, ${df_top_ten.at[1,'PRICE']:,}, or ${df_top_ten.at[1,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[1,'list_agent_name_1']} with {df_top_ten.at[1,'list_broker_name_1']}, and {df_top_ten.at[1,'list_agent_name_2']} with {df_top_ten.at[1,'list_broker_name_2']} had the listing.
{df_top_ten.at[1,'buy_agent_name_1']} with {df_top_ten.at[1,'buy_broker_name_1']}, and {df_top_ten.at[1,'buy_agent_name_2']} with {df_top_ten.at[1,'buy_broker_name_2']} represented the buyer.

<figure>
 <div class="container">
   <div class="iframe-wrap">
   <iframe src="{final_name}" width="100%" height="600" frameBorder="0" scrolling="no"></iframe>
  </div>
</div>
  <figcaption align="right"><a href="https://leafletjs.com/">Leaflet</a> map created by Adam Farence | Data by © <a href="https://www.openstreetmap.org/#map=4/38.01/-95.84"> OpenStreetMap</a>, under <a href="https://www.openstreetmap.org/copyright">ODbl.</a></figcaption>
</figure>

Here’s a breakdown of the top 10 sales from {week_starting} to {week_ending}:
'''

print(story_string)


HED: 321 OCEAN CONDO closing tops Miami-Dade weekly condo sales 
DEK: Top 10 sales ranged from $2,575,000 to $11,250,000
FEATURED HED:
SEO HED: Miami-Dade County Weekly Condo Report 
SEO DESCRIPTION: Miami-Dade County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Miami-Dade condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Miami-Dade, weekly condo sales, Key Biscayne, Miami Beach, Miami, Sunny Isles Beach, condos, condo market

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Miami-Dade County’s NEWS PEG HERE.

Brokers closed 126 condo sales totaling $106,325,153 million from Mar. 23rd to Mar. 30th. The previous week, brokers closed 116 condo sales totaling $79,205,150.

Last week’s units sold for an average of $843,850, higher than the $682,803 average price from the p

In [50]:
story_checklist = '''
\033[1mRemember to...:\033[0m

1. Double check all names. Sometimes names differ between the hed and the body of the story.
    For example, "Surf Club Four Seasons" in the HED and "Four Seasons Residences at the Surfclub"
    in the body.
    
2. Add in context, if available. When there's a high-priced condo sale, check and see if there's
    a story. If there is, add in some extra details and link back to it.
'''

In [51]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

In [52]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [53]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-03-24,Condo/Co-op,321 Ocean Dr #201,Miami Beach,FL,33139.0,11250000.0,5.0,5.5,321 OCEAN CONDO,4400.0,NaN,2015.0,NaN,2557.0,28702.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/321-Ocea...,MARMLS,A11695945,N,Y,25.772296,-80.131901,https://www.redfin.com/FL/Miami-Beach/321-Ocea...,Michael Duchon,The Corcoran Group,Eloy Carmenate,The Corcoran Group,Darin Tansey,Douglas Elliman,None,None,1,orange,321 OCEAN CONDO at 321 Ocean Dr #201 in Miami ...
1,PAST SALE,2025-03-28,Condo/Co-op,17749 Collins Ave #802,Sunny Isles Beach,FL,33160.0,6100000.0,4.0,6.5,17749 COLLINS AVENUE COND,4609.0,NaN,2015.0,NaN,1323.0,10288.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11611470,N,Y,25.940065,-80.120264,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,Adi Amuial,Miles Goldstein Real Estate,None,None,Adriana Vargas Hernandez,Acqualina Realty,None,None,2,blue,17749 COLLINS AVENUE COND at 17749 Collins Ave...
2,PAST SALE,2025-03-28,Condo/Co-op,17141 Collins Ave #1501,Sunny Isles Beach,FL,33160.0,4950000.0,3.0,3.5,MUSE CONDO,3195.0,NaN,2018.0,NaN,1549.0,9176.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11490517,N,Y,25.934939,-80.120918,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,Roberta Poli,"Related ISG Realty, LLC.",None,None,Viktoriya Donisova Ho,"Compass Florida, LLC.",None,None,3,blue,MUSE CONDO at 17141 Collins Ave #1501 in Sunny...
3,PAST SALE,2025-03-24,Condo/Co-op,18975 Collins Ave #2303,Sunny Isles Beach,FL,33160.0,3800000.0,3.0,3.5,18975 Collins Condo,2271.0,NaN,2019.0,NaN,1673.0,3734.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,Beaches MLS,RX-11004551,N,Y,25.951740,-80.119755,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,Karine Frangulyan,Signature One Luxury Estates LLC,None,None,Wendy Cohen,One Sotheby's International Realty,None,None,4,blue,18975 Collins Condo at 18975 Collins Ave #2303...
4,PAST SALE,2025-03-26,Condo/Co-op,18975 Collins Ave #903,Sunny Isles Beach,FL,33160.0,3775000.0,3.0,3.5,18975 COLLINS CONDO,2271.0,NaN,2019.0,NaN,1662.0,8904.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,MARMLS,A11712041,N,Y,25.951740,-80.119755,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,Roger Silva,The Florida Lounge Realty LLC,None,None,Romina Grinberg,"Compass Florida, LLC",None,None,5,blue,18975 COLLINS CONDO at 18975 Collins Ave #903 ...
5,PAST SALE,2025-03-27,Condo/Co-op,300 S Pointe Dr #1105,Miami Beach,FL,33139.0,3600000.0,3.0,2.5,PORTOFINO TOWER CONDO,2340.0,NaN,1996.0,NaN,1538.0,4016.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/300-S-Po...,MARMLS,A11710839,N,Y,25.767706,-80.134877,https://www.redfin.com/FL/Miami-Beach/300-S-Po...,None,None,None,None,None,None,None,None,6,blue,PORTOFINO TOWER CONDO at 300 S Pointe Dr #1105...
6,PAST SALE,2025-03-24,Condo/Co-op,2669 S Bayshore Dr Unit 1602N,Miami,FL,33133.0,3500000.0,3.0,3.5,THE GROVE AT GRAND BAY,1829.0,NaN,2016.0,NaN,1914.0,3083.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2669-S-Bayshor...,MARMLS,A11582132,N,Y,25.730676,-80.237117,https://www.redfin.com/FL/Miami/2669-S-Bayshor...,Ivan Madera,Fortune Christie's International Real Estate,None,None,Lindsey Lamchick,The Edit,None,None,7,blue,THE GROVE AT GRAND BAY at 2669 S Bayshore Dr U...
7,PAST SALE,2025-03-28,Condo/Co-op,1800 Sunset Harbour Dr Unit 1102/04,Miami Beach,FL,33139.0,2837500.0,4.0,3.5,SUNSET HARBOUR SOUTH COND,2246.0,NaN,1998.0,NaN,1263.0,3317.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/1800-Sun...,MARMLS,A11606692,N,Y,2

In [54]:
print(story_string)

# Highest and lowest sale price
highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

print(f"{ME}{BR}{df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[highest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_2']}\n")
print(f"{LE}{BR}{df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_2']}\n")

# Highest and lowest psf
highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

print(f"{MAX_PSF}{BR}{df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[highest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_2']}\n")
print(f"{MIN_PSF}{BR}{df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_2']}\n")


print("\n--------SALE LINKS FOR REFERENCE ONLY--------\n")
print('Top sale: ',df_top_ten['URL'].iloc[0])
print('Second priciest sale: ', df_top_ten['URL'].iloc[1])
print('Cheapest top ten sale: ',df_top_ten['URL'].iloc[-1])
print('Second Cheapest top ten sale: ',df_top_ten['URL'].iloc[-2])
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print('Highest top ten psf: ',max_url)

# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print('Lowest top ten psf: ', min_url)


HED: 321 OCEAN CONDO closing tops Miami-Dade weekly condo sales 
DEK: Top 10 sales ranged from $2,575,000 to $11,250,000
FEATURED HED:
SEO HED: Miami-Dade County Weekly Condo Report 
SEO DESCRIPTION: Miami-Dade County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Miami-Dade condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Miami-Dade, weekly condo sales, Key Biscayne, Miami Beach, Miami, Sunny Isles Beach, condos, condo market

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Miami-Dade County’s NEWS PEG HERE.

Brokers closed 126 condo sales totaling $106,325,153 million from Mar. 23rd to Mar. 30th. The previous week, brokers closed 116 condo sales totaling $79,205,150.

Last week’s units sold for an average of $843,850, higher than the $682,803 average price from the p

In [55]:
print('Top sale:', df_top_ten['URL'].iloc[0])

Top sale: https://www.redfin.com/FL/Miami-Beach/321-Ocean-Dr-33139/unit-201/home/45471133


In [56]:
print(df_top_ten['URL'].iloc[1])

https://www.redfin.com/FL/Sunny-Isles-Beach/17749-Collins-Ave-33160/unit-802/home/110130747


In [57]:
print(df_top_ten['URL'].iloc[-1])

https://www.redfin.com/FL/Key-Biscayne/749-Crandon-Blvd-33149/unit-212/home/42923730


In [58]:
print(df_top_ten['URL'].iloc[-2])

https://www.redfin.com/FL/Sunny-Isles-Beach/18975-Collins-Ave-33160/unit-604/home/171050928


In [59]:
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print(max_url)

https://www.redfin.com/FL/Miami-Beach/321-Ocean-Dr-33139/unit-201/home/45471133


In [60]:
# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print(min_url)

https://www.redfin.com/FL/Key-Biscayne/749-Crandon-Blvd-33149/unit-212/home/42923730


In [61]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-03-24,Condo/Co-op,321 Ocean Dr #201,Miami Beach,FL,33139.0,11250000.0,5.0,5.5,321 OCEAN CONDO,4400.0,NaN,2015.0,NaN,2557.0,28702.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/321-Ocea...,MARMLS,A11695945,N,Y,25.772296,-80.131901,https://www.redfin.com/FL/Miami-Beach/321-Ocea...,Michael Duchon,The Corcoran Group,Eloy Carmenate,The Corcoran Group,Darin Tansey,Douglas Elliman,None,None,1,orange,321 OCEAN CONDO at 321 Ocean Dr #201 in Miami ...
1,PAST SALE,2025-03-28,Condo/Co-op,17749 Collins Ave #802,Sunny Isles Beach,FL,33160.0,6100000.0,4.0,6.5,17749 COLLINS AVENUE COND,4609.0,NaN,2015.0,NaN,1323.0,10288.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11611470,N,Y,25.940065,-80.120264,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,Adi Amuial,Miles Goldstein Real Estate,None,None,Adriana Vargas Hernandez,Acqualina Realty,None,None,2,blue,17749 COLLINS AVENUE COND at 17749 Collins Ave...
2,PAST SALE,2025-03-28,Condo/Co-op,17141 Collins Ave #1501,Sunny Isles Beach,FL,33160.0,4950000.0,3.0,3.5,MUSE CONDO,3195.0,NaN,2018.0,NaN,1549.0,9176.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11490517,N,Y,25.934939,-80.120918,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,Roberta Poli,"Related ISG Realty, LLC.",None,None,Viktoriya Donisova Ho,"Compass Florida, LLC.",None,None,3,blue,MUSE CONDO at 17141 Collins Ave #1501 in Sunny...
3,PAST SALE,2025-03-24,Condo/Co-op,18975 Collins Ave #2303,Sunny Isles Beach,FL,33160.0,3800000.0,3.0,3.5,18975 Collins Condo,2271.0,NaN,2019.0,NaN,1673.0,3734.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,Beaches MLS,RX-11004551,N,Y,25.951740,-80.119755,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,Karine Frangulyan,Signature One Luxury Estates LLC,None,None,Wendy Cohen,One Sotheby's International Realty,None,None,4,blue,18975 Collins Condo at 18975 Collins Ave #2303...
4,PAST SALE,2025-03-26,Condo/Co-op,18975 Collins Ave #903,Sunny Isles Beach,FL,33160.0,3775000.0,3.0,3.5,18975 COLLINS CONDO,2271.0,NaN,2019.0,NaN,1662.0,8904.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,MARMLS,A11712041,N,Y,25.951740,-80.119755,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,Roger Silva,The Florida Lounge Realty LLC,None,None,Romina Grinberg,"Compass Florida, LLC",None,None,5,blue,18975 COLLINS CONDO at 18975 Collins Ave #903 ...
5,PAST SALE,2025-03-27,Condo/Co-op,300 S Pointe Dr #1105,Miami Beach,FL,33139.0,3600000.0,3.0,2.5,PORTOFINO TOWER CONDO,2340.0,NaN,1996.0,NaN,1538.0,4016.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/300-S-Po...,MARMLS,A11710839,N,Y,25.767706,-80.134877,https://www.redfin.com/FL/Miami-Beach/300-S-Po...,None,None,None,None,None,None,None,None,6,blue,PORTOFINO TOWER CONDO at 300 S Pointe Dr #1105...
6,PAST SALE,2025-03-24,Condo/Co-op,2669 S Bayshore Dr Unit 1602N,Miami,FL,33133.0,3500000.0,3.0,3.5,THE GROVE AT GRAND BAY,1829.0,NaN,2016.0,NaN,1914.0,3083.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2669-S-Bayshor...,MARMLS,A11582132,N,Y,25.730676,-80.237117,https://www.redfin.com/FL/Miami/2669-S-Bayshor...,Ivan Madera,Fortune Christie's International Real Estate,None,None,Lindsey Lamchick,The Edit,None,None,7,blue,THE GROVE AT GRAND BAY at 2669 S Bayshore Dr U...
7,PAST SALE,2025-03-28,Condo/Co-op,1800 Sunset Harbour Dr Unit 1102/04,Miami Beach,FL,33139.0,2837500.0,4.0,3.5,SUNSET HARBOUR SOUTH COND,2246.0,NaN,1998.0,NaN,1263.0,3317.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/1800-Sun...,MARMLS,A11606692,N,Y,2

## Time on Market Calculator

In [63]:
################ YEAR, MONTH, DAY #######################

date1 = datetime(2024, 6, 22) ## List (Earlier) date
date2 = datetime(2025, 3, 28) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

279


## Clean CSV for Datawrapper Chart

In [64]:
chart_df = df_top_ten

In [65]:
chart_df = chart_df.fillna(" ")

In [66]:
chart_df['LISTING INFO'] = chart_df['list_agent_name_1'] + ' ' + chart_df['list_broker_name_1'] + ' ' + chart_df['list_agent_name_2'] + ' ' + chart_df['list_broker_name_2'] 
chart_df['BUYING INFO'] = chart_df['buy_agent_name_1'] + ' ' + chart_df['buy_broker_name_1'] + ' ' + chart_df['buy_agent_name_2'] + ' ' + chart_df['buy_broker_name_2'] 


In [67]:
chart_df['ADDRESS'] = chart_df['ADDRESS'] + ' in ' + chart_df['CITY']

In [68]:
columns_to_drop = [
    "SALE TYPE",
    "PROPERTY TYPE",
    "STATE OR PROVINCE",
    "ZIP OR POSTAL CODE",
    "LOT SIZE",
    "DAYS ON MARKET",
    "HOA/MONTH",
    "STATUS",
    "NEXT OPEN HOUSE START TIME",
    "NEXT OPEN HOUSE END TIME",
    "URL",
    "FAVORITE",
    "INTERESTED",
    "LATITUDE",
    "LONGITUDE",
    "url",
    "RANK",
    "COLOR",
    "FULL_NAME",
    "list_agent_name_1",
    "list_broker_name_1",
    "list_agent_name_2",
    "list_broker_name_2",
    "buy_agent_name_1",
    "buy_broker_name_1",
    "buy_agent_name_2",
    "buy_broker_name_2",
    "CITY",
    "LOCATION"
]

In [69]:
chart_df = chart_df.drop(columns=columns_to_drop)

In [70]:
chart_df['$/SQUARE FEET'].astype(float)

0    2557.0
1    1323.0
2    1549.0
3    1673.0
4    1662.0
5    1538.0
6    1914.0
7    1263.0
8    1709.0
9     972.0
Name: $/SQUARE FEET, dtype: float64

In [71]:
chart_df['PRICE'] = chart_df['PRICE'].apply(lambda x: "${:,.0f}".format(x))
chart_df['$/SQUARE FEET'] = chart_df['$/SQUARE FEET'].apply(lambda x: "${:,.0f}".format(x) if pd.notnull(x) else x)
chart_df['SQUARE FEET'] = chart_df['SQUARE FEET'].apply(lambda x: "{:,.0f}".format(x))
chart_df['YEAR BUILT'] = chart_df['YEAR BUILT'].apply(lambda x: "{:.0f}".format(x))
chart_df['BEDS'] = chart_df['BEDS'].apply(lambda x: "{:.0f}".format(x))

In [72]:
csv_date_string = today.strftime("%m_%d_%Y")

In [73]:
chart_df.to_csv(f"WCS_Chart_{csv_date_string}.csv",index=False)

In [74]:
chart_df

,SOLD DATE,ADDRESS,PRICE,BEDS,BATHS,SQUARE FEET,YEAR BUILT,$/SQUARE FEET,SOURCE,MLS#,LISTING INFO,BUYING INFO
0,2025-03-24,321 Ocean Dr #201 in Miami Beach,"$11,250,000",5,5.5,"4,400",2015,"$2,557",MARMLS,A11695945,Michael Duchon The Corcoran Group Eloy Carmena...,Darin Tansey Douglas Elliman
1,2025-03-28,17749 Collins Ave #802 in Sunny Isles Beach,"$6,100,000",4,6.5,"4,609",2015,"$1,323",MARMLS,A11611470,Adi Amuial Miles Goldstein Real Estate,Adriana Vargas Hernandez Acqualina Realty
2,2025-03-28,17141 Collins Ave #1501 in Sunny Isles Beach,"$4,950,000",3,3.5,"3,195",2018,"$1,549",MARMLS,A11490517,"Roberta Poli Related ISG Realty, LLC.","Viktoriya Donisova Ho Compass Florida, LLC."
3,2025-03-24,18975 Collins Ave #2303 in Sunny Isles Beach,"$3,800,000",3,3.5,"2,271",2019,"$1,673",Beaches MLS,RX-11004551,Karine Frangulyan Signature One Luxury Estates...,Wendy Cohen One Sotheby's International Realty...
4,2025-03-26,18975 Collins Ave #903 in Sunny Isles Beach,"$3,775,000",3,3.5,"2,271",2019,"$1,662",MARMLS,A11712041,Roger Silva The Florida Lounge Realty LLC,"Romina Grinberg Compass Florida, LLC"
5,2025-03-27,300 S Pointe Dr #1105 in Miami Beach,"$3,600,000",3,2.5,"2,340",1996,"$1,538",MARMLS,A11710839,,
6,2025-03-24,2669 S Bayshore Dr Unit 1602N in Miami,"$3,500,000",3,3.5,"1,829",2016,"$1,914",MARMLS,A11582132,Ivan Madera Fortune Christie's International R...,Lindsey Lamchick The Edit
7,2025-03-28,1800 Sunset Harbour Dr Unit 1102/04 in Miami B...,"$2,837,500",4,3.5,"2,246",1998,"$1,263",MARMLS,A11606692,"Paul Sasseville Compass Florida, LLC.",Cory Waldman Coldwell Banker Realty
8,2025-03-24,18975 Collins Ave #604 in Sunny Isles Beach,"$2,600,000",2,2.5,"1,521",2019,"$1,709",MARMLS,A11720744,Luciene Cofresi Fortune International Realty,Claudio Minones One Sotheby's International Re...
9,2025-03-25,749 Crandon Blvd #212 in Key Biscayne,"$2,575,000",4,3.0,"2,650",1998,$972,MARMLS,A11618404,Gloria Ibarra Realty Progress LLC,Natalia Salgado DLS International Realty
